In [1]:
import pandas as pd
import numpy as np
import os.path
from os import path
import matplotlib.pyplot as plt
import scipy.stats as stats


In [2]:
# filename = 'datafile.txt'
# pd.read_csv(filename, sep='\t')

In [3]:
df=pd.read_csv("Data_211118_0901.CSV")
df.head()

,Date,Time,TE-007 R.T.,CA O2 25,DP (600),BPR Equilibar,DP (300),CA O2 100,FT-201new,DP (600).1,...,V-406,V-501,V-502,V-503,V-504,V-505,V-506,V-601,V-602,Notes
0,11/18/2021,09:01:57,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,NaN
1,11/18/2021,09:01:58,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,NaN
2,11/18/2021,09:01:59,21.76651,10.395285,0.004395,0.21973,0.0,95.033188,0.017423,0.004395,...,0,0,0,0,0,0,0,0,0,NaN
3,11/18/2021,09:02:00,21.76651,10.406271,0.000000,0.21973,0.0,94.945297,0.017423,0.004395,...,0,0,0,0,0,0,0,0,0,NaN
4,11/18/2021,09:02:01,21.76651,10.406271,0.004395,0.21973,0.0,94.945297,-0.000244,0.000000,...,0,0,0,0,0,0,0,0,0,NaN


In [4]:
type(df["Time"][0])

str

In [5]:
df['Diff'] = pd.to_datetime(df['Time'],format= '%H:%M:%S').diff().dt.total_seconds()
df['Time1'] = pd.to_datetime(df['Time'],format= '%H:%M:%S').diff()
DeltaTime=df['Diff'].drop([0])


FT_101=df['FT-101'].drop(len(df)-1)
FT_102=df['FT-101'].drop(len(df)-1)
FT_201=df['FT-201'].drop(len(df)-1)
FT_301=df['FT-301'].drop(len(df)-1)
FT_401=df['FT-401'].drop(len(df)-1)
FT_501=df['FT-501'].drop(len(df)-1)
FT_601=df['FT-601'].drop(len(df)-1)

PT_101=df['PT-101'].drop(len(df)-1)
PT_201=df['PT-201'].drop(len(df)-1)
PT_301=df['PT-301'].drop(len(df)-1)
PT_401=df['PT-401'].drop(len(df)-1)
PT_501=df['PT-501'].drop(len(df)-1)
PT_601=df['PT-601'].drop(len(df)-1)

CA_O2_100=df[' CA O2 100'].drop(len(df)-1)
CA_O2_25=df['CA O2 25'].drop(len(df)-1)
BGA=df['BGA (200)'].drop(len(df)-1)


DeltaTime=DeltaTime.reset_index(drop=True)
FT_101=FT_101.reset_index(drop=True)
FT_102=FT_102.reset_index(drop=True)
FT_201=FT_201.reset_index(drop=True)
FT_301=FT_301.reset_index(drop=True)
FT_401=FT_401.reset_index(drop=True)
FT_501=FT_501.reset_index(drop=True)
FT_601=FT_601.reset_index(drop=True)

PT_101=PT_101.reset_index(drop=True)
PT_201=PT_201.reset_index(drop=True)
PT_301=PT_301.reset_index(drop=True)
PT_401=PT_401.reset_index(drop=True)
PT_501=PT_501.reset_index(drop=True)
PT_601=PT_601.reset_index(drop=True)

CA_O2_100=CA_O2_100.reset_index(drop=True)
CA_O2_25=CA_O2_25.reset_index(drop=True)
BGA=BGA.reset_index(drop=True)

## Mass flow balance

In [41]:
#Time region
StartTime='10:05:00'
EndTime='10:15:00'
#Parameters
O2ambient=0.21
N2ambient=1-O2ambient
N2UHP=1.0
Pambient=12.2
Psea=14.696
Start=3555
End=Start+120
# Start=df[df['Time'] == StartTime].index.values[0]
# End=df[df['Time'] == EndTime].index.values[0]
print("Start:" + str(Start), " End:" + str(End))

Start:3555  End:3675


In [81]:
O2_Purge_Inlet=(FT_301[Start:End]/60)*CA_O2_100[Start:End]/100
O2_Purge_Outlet=(FT_401[Start:End]/60)*CA_O2_100[Start:End]/100
O2_Feed_Inlet=(FT_102[Start:End]/60)*O2ambient
O2_Product_Outlet=(FT_601[Start:End]/60)*CA_O2_25[Start:End]/100
O2_Blowdown_Outlet=(FT_201[Start:End]/60)*BGA[Start:End]/100
N2_Repress_Inlet=(FT_501[Start:End]/60)*N2UHP
N2_Feed_Inlet=(FT_102[Start:End]/60)*N2ambient
N2_Product_Outlet=(FT_601[Start:End]/60)*(100-CA_O2_25[Start:End])/100
N2_Purge_Outlet=(FT_401[Start:End]/60)*(100-CA_O2_100[Start:End])/100
N2_Blowdown_Outlet=(FT_401[Start:End]/60)*(100-BGA[Start:End])/100

In [82]:
O2_In=O2_Purge_Inlet.sum(axis=0)+O2_Feed_Inlet.sum(axis=0)
O2_Out=O2_Purge_Outlet.sum(axis=0)+O2_Product_Outlet.sum(axis=0)+O2_Blowdown_Outlet.sum(axis=0)
O2massbalance=O2_In-O2_Out

In [83]:
N2_In=N2_Repress_Inlet.sum(axis=0)+N2_Feed_Inlet.sum(axis=0)
N2_Out=N2_Purge_Outlet.sum(axis=0)+N2_Product_Outlet.sum(axis=0)+N2_Blowdown_Outlet.sum(axis=0)
N2massbalance=N2_In-N2_Out

In [86]:
O2Purge_In=O2_Purge_Outlet.sum(axis=0)
O2Blowdown_Out=O2_Blowdown_Outlet.sum(axis=0)
O2purgebalance=O2Blowdown_Out-O2Purge_In

In [87]:
print("O2 mass balance:" + str(O2massbalance), " N2 mass balance:" + str(N2massbalance)," Purge mass balance:" + str(O2purgebalance),)

O2 mass balance:-2.111738045407332  N2 mass balance:-0.22892848497994778  Purge mass balance:-0.39115508427274825


## Creating the plots

In [64]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20/2.54, 10/2.54] 
fig = plt.figure()
ax = plt.axes()
line1,=plt.plot(O2_Purge_Outlet, color='Indigo', linestyle='-', linewidth=3,label='O2purge L')
line2,=plt.plot(O2_Blowdown_Outlet, color='blue', linestyle='-', linewidth=3,label='O2Blowdown L')
plt.legend(handles=[line1,line2], loc='lower right')

plt.xlabel('Seconds', fontsize=15)
plt.ylabel('O2 liters', fontsize=15)
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
plt.savefig('O2purge.png')

In [62]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.rcParams['figure.figsize'] = [20/2.54, 10/2.54] 

line1,=ax1.plot(CA_O2_100[Start:End], color='red', linestyle='-', linewidth=3,label='O2purge %VOL')
line2,=ax1.plot(BGA[Start:End], color='green', linestyle='-', linewidth=3,label='O2Blowdown %VOL')
line3,=ax2.plot(PT_201[Start:End], color='indigo', linestyle='-', linewidth=3,label='PT201')
line4,=ax2.plot(PT_401[Start:End], color='blue', linestyle='-', linewidth=3,label='PT401')
plt.legend(handles=[line1,line2,line3,line4], loc='lower right')

ax1.set_xlabel('Seconds', fontsize=15)
ax1.set_ylabel('O2%VOL', fontsize=15)
ax2.set_ylabel('psia', fontsize=15)
plt.grid()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [20/2.54, 10/2.54] 
ax2 = ax1.twinx()
line1,=ax1.plot(O2_Purge_Inlet, color='Indigo', linestyle='-', linewidth=3,label='O2purge L')
line2,=ax1.plot(O2_Blowdown_Outlet, color='blue', linestyle='-', linewidth=3,label='O2Blowdown L')
line3,=ax2.plot(FT_301[Start:End], color='green', linestyle='-', linewidth=3,label='FT301')
line4,=ax2.plot(FT_401[Start:End], color='red', linestyle='-', linewidth=3,label='FT401')
plt.legend(handles=[line1,line2,line3,line4], loc='lower right')

ax1.set_xlabel('Seconds', fontsize=15)
ax1.set_ylabel('O2 Liters', fontsize=15)
ax2.set_ylabel('O2 LPM', fontsize=15)
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Interactive Plot

In [53]:
%matplotlib widget
fig.canvas.toolbar_visible = True
fig.canvas.header_visible = True
fig.canvas.resizable = True

#Defualt user interface
#%matplotlib inline

In [65]:
fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [20/2.54, 10/2.54] 
ax2 = ax1.twinx()
line1,=ax1.plot(O2_Purge_Inlet, color='Indigo', linestyle='-', linewidth=3,label='O2purge L')
line2,=ax1.plot(O2_Blowdown_Outlet, color='blue', linestyle='-', linewidth=3,label='O2Blowdown L')
line3,=ax2.plot(FT_301[Start:End], color='green', linestyle='-', linewidth=3,label='FT301')
line4,=ax2.plot(FT_401[Start:End], color='red', linestyle='-', linewidth=3,label='FT401')
plt.legend(handles=[line1,line2,line3,line4], loc='lower right')

ax1.set_xlabel('Seconds', fontsize=15)
ax1.set_ylabel('O2 Liters', fontsize=15)
ax2.set_ylabel('O2 LPM', fontsize=15)
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
df.head()
#df.plot(kind="line")

,Date,Time,TE-007 R.T.,CA O2 25,DP (600),BPR Equilibar,DP (300),CA O2 100,FT-201new,DP (600).1,...,V-502,V-503,V-504,V-505,V-506,V-601,V-602,Notes,Diff,Time1
0,11/18/2021,09:01:57,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,NaN,NaN,NaT
1,11/18/2021,09:01:58,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,NaN,1.0,0 days 00:00:01
2,11/18/2021,09:01:59,21.76651,10.395285,0.004395,0.21973,0.0,95.033188,0.017423,0.004395,...,0,0,0,0,0,0,0,NaN,1.0,0 days 00:00:01
3,11/18/2021,09:02:00,21.76651,10.406271,0.000000,0.21973,0.0,94.945297,0.017423,0.004395,...,0,0,0,0,0,0,0,NaN,1.0,0 days 00:00:01
4,11/18/2021,09:02:01,21.76651,10.406271,0.004395,0.21973,0.0,94.945297,-0.000244,0.000000,...,0,0,0,0,0,0,0,NaN,1.0,0 days 00:00:01
